<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

e:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
e:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Instal

In [2]:
# Cargar datos
path = "C:/Users/USER/OneDrive - Fundación Universitaria Konrad Lorenz/nlp/Taller9/reviews_vidjew_es.csv"
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [11]:
dictionary = Dictionary(data['Pre-Processed'].values)
print (dictionary)

Dictionary(3683 unique tokens: ['buen', 'buena', 'calidad', 'presentación', 'casa']...)


In [10]:
dictionary.filter_extremes(no_below=5, no_above=0.5) 
print (dictionary)

Dictionary(505 unique tokens: ['buen', 'buena', 'calidad', 'presentación', 'casa']...)


In [12]:
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]
print (corpus)

[[(0, 1), (1, 2), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(5, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(12, 1), (15, 1), (22, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)], [(34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)], [(2, 1), (3, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)], [(7, 3), (27, 1), (40, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1)], [(2, 1), (15, 1), (30, 1), (44, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1)], [(6, 1), (30, 1)

In [14]:
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 10, passes=50)

In [15]:
print (model)

LdaModel(num_terms=3683, num_topics=10, decay=0.5, chunksize=2000)


In [16]:
#Resultados
model.print_topics(num_words=10)

[(0,
  '0.014*"producto" + 0.014*"bien" + 0.013*"perfecto" + 0.011*"juego" + 0.009*"precio" + 0.008*"si" + 0.006*"buen" + 0.006*"bonitos" + 0.006*"aunque" + 0.006*"mando"'),
 (1,
  '0.025*"bien" + 0.016*"calidad" + 0.014*"buena" + 0.010*"precio" + 0.010*"producto" + 0.009*"regalo" + 0.008*"llegado" + 0.006*"juego" + 0.006*"pedido" + 0.005*"bastante"'),
 (2,
  '0.013*"bien" + 0.011*"si" + 0.010*"mas" + 0.010*"jugar" + 0.009*"juego" + 0.007*"calidad" + 0.007*"dos" + 0.006*"llegado" + 0.005*"mandos" + 0.005*"tiempo"'),
 (3,
  '0.013*"bien" + 0.011*"precio" + 0.009*"si" + 0.007*"juego" + 0.007*"bonitos" + 0.006*"mando" + 0.006*"calidad" + 0.005*"bastante" + 0.005*"dos" + 0.005*"mal"'),
 (4,
  '0.013*"buen" + 0.010*"pendientes" + 0.010*"calidad" + 0.009*"llegado" + 0.008*"pequeños" + 0.008*"mas" + 0.007*"producto" + 0.007*"foto" + 0.007*"bonito" + 0.007*"pequeño"'),
 (5,
  '0.015*"bonito" + 0.013*"si" + 0.009*"regalo" + 0.009*"calidad" + 0.008*"foto" + 0.008*"juego" + 0.007*"bien" + 0.007*"

###  `[25 pts]` Punto 3: Visualización de LDA

In [17]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)

In [18]:
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

In [22]:
##1 Se puede apreciar la evolución con las herramientas, llegando al punto (con el actual taller) de poder entender en detalle el impacto de los términos sobre las categorías

In [20]:
##2 Es necesario construir el camino desde lo básico, para poder "sacar el jugo" de los temas más avanzados. Sin embargo, aplicaciones más complejas requerirán un mayor nivel de experiencia, que es el llamado a seguir

In [21]:
##3 No pensé volver a encontrarme con temas de geometría (históricamente hemos estado en pelea desde siempre) :S